<a href="https://colab.research.google.com/github/LeechXDD/9417_Pro_Project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://www.kaggle.com/code/gusthema/student-performance-w-tensorflow-decision-forests

In [ ]:
!pip install tensorflow_addons
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import f1_score
import tensorflow_addons as tfa

##Data preprocessing

In [ ]:
def get_minimal_dtype(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [ ]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_df = pd.read_csv('/content/drive/MyDrive/9417project/predict-student-performance-from-game-play.zip (Unzipped Files)/train.csv', dtype=dtypes)
dataset_df = get_minimal_dtype(dataset_df)
labels = pd.read_csv('/content/drive/MyDrive/9417project/predict-student-performance-from-game-play.zip (Unzipped Files)/train_labels.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memory usage of dataframe is 1529.83 MB
Memory usage became:  1053.3384094238281  MB


In [ ]:
# Find out columns with missing values
missing_values = dataset_df.isnull().sum()

# Fill missing values
dataset_df['page'].fillna(-1, inplace=True)
dataset_df['room_coor_x'].fillna(-1, inplace=True)
dataset_df['room_coor_y'].fillna(-1, inplace=True)
dataset_df['screen_coor_x'].fillna(-1, inplace=True)
dataset_df['screen_coor_y'].fillna(-1, inplace=True)
dataset_df['hover_duration'].fillna(-1, inplace=True)

# Cap 'elapsed_time' at the 99th percentile
dataset_df['elapsed_time'] = dataset_df['elapsed_time'].clip(upper=dataset_df['elapsed_time'].quantile(0.99))

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [ ]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
BINNING = ['elapsed_time', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']

from sklearn.preprocessing import PowerTransformer

def feature_engineer(dataset_df):
    dfs = []
    pt = PowerTransformer(method='yeo-johnson')

    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = c + '_nunique'
        dfs.append(tmp)

        # Create dummy variables for top N most common events and names
        top_N = dataset_df[c].value_counts()[:10].index
        for val in top_N:
            dataset_df[c + '_' + val] = (dataset_df[c] == val).astype(int)
        tmp = dataset_df.groupby(['session_id','level_group']).agg({c + '_' + val: 'sum' for val in top_N})
        dfs.append(tmp)

    for c in NUMERICAL:
        # Fill missing values with the column median
        dataset_df[c].fillna(dataset_df[c].median(), inplace=True)

        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = c + '_mean'
        dfs.append(tmp)

        # Compute standard deviation only for certain features
        if c in BINNING:
            tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
            tmp.name = c + '_std'
            dfs.append(tmp)

        # Normalize 'elapsed_time' column
        if c == 'elapsed_time':
            dataset_df[c] = pt.fit_transform(dataset_df[[c]])

        # Binning
        if c in BINNING:  # Check if column is in the list of columns to bin
            dataset_df[c+'_bin'] = pd.qcut(dataset_df[c], q=4, duplicates='drop')
            #dataset_df[c+'_bin'] = pd.qcut(dataset_df[c], q=4, duplicates='drop').astype('category')

            tmp = dataset_df.groupby(['session_id','level_group'])[c+'_bin'].agg('count')
            tmp.name = c + '_bin_count'
            dfs.append(tmp)

    # Interaction between screen coordinates
    if 'screen_coor_x' in NUMERICAL and 'screen_coor_y' in NUMERICAL:
        # Compute Euclidean distance instead of product
        dataset_df['screen_coor'] = np.sqrt(dataset_df['screen_coor_x']**2 + dataset_df['screen_coor_y']**2)
        tmp = dataset_df.groupby(['session_id','level_group'])['screen_coor'].agg(['mean', 'std'])
        tmp.columns = ['screen_coor_mean', 'screen_coor_std']
        dfs.append(tmp)

    # Aggregated features
    if 'hover_duration' in NUMERICAL:
        dataset_df['total_hover_duration'] = dataset_df.groupby(['session_id'])['hover_duration'].transform('sum')
        tmp = dataset_df.groupby(['session_id','level_group'])['total_hover_duration'].agg('mean')
        tmp.name = 'total_hover_duration_mean'
        dfs.append(tmp)

    dataset_df = pd.concat(dfs, axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')

    return dataset_df

In [ ]:
def partial_loader(df):
  split = 5
  epoch_length = df.shape[0]//split
  start = 0
  partial_df = df.iloc[start:start+epoch_length].copy(deep = True)
  partial_df = feature_engineer(partial_df)
  start += epoch_length
  for i in range(split - 2):
    partial_df = pd.concat([partial_df,feature_engineer(df.iloc[start:start+epoch_length].copy(deep = True))])
    start += epoch_length
  partial_df = pd.concat([partial_df,feature_engineer(df.iloc[start:])])
  return partial_df

In [ ]:
dataset_df = partial_loader(dataset_df)

<ipython-input-38-351aa74f6fdf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df[c + '_' + val] = (dataset_df[c] == val).astype(int)
<ipython-input-38-351aa74f6fdf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df[c + '_' + val] = (dataset_df[c] == val).astype(int)
<ipython-input-38-351aa74f6fdf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [ ]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)

In [ ]:
# Fetch the unique list of user sessions in the validation dataset. We assigned
# `session_id` as the index of our feature engineered dataset. Hence fetching
# the unique values in the index column will give us a list of users in the
# validation set.
VALID_USER_LIST = valid_x.index.unique()

# Create a dataframe for storing the predictions of each question for all users
# in the validation set.
# For this, the required size of the data frame is:
# (no: of users in validation set  x no of questions).
# We will initialize all the predicted values in the data frame to zero.
# The dataframe's index column is the user `session_id`s.
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Create an empty dictionary to store the models created for each question.
models = {}

f1_scores = {}

# Create an empty dictionary to store the evaluation score for each question.
evaluation_dict ={}

In [ ]:
def prepare_data(df, sequence_length):

    # Prepare data for LSTM, based on sequence length.

    # Get the last value of correct column
    labels = df.groupby(df.index)["correct"].last().values

    # Drop the 'correct' and 'level_group' columns from the features to provent target leak
    features = df.drop(columns=["correct", "level_group"])

    data = []
    label = []
    users = []  # Add a list to store the users

    # Create sequences
    for user, group in features.groupby(features.index):
        user_features = group.values
        if len(user_features) < sequence_length:
            padding = np.zeros((sequence_length - len(user_features), user_features.shape[1])) #append zero if features is less than sequence length
            user_features = np.concatenate((user_features, padding)) # combine together
        data.append(user_features)
        label.append(labels[len(data)-1])
        users.append(user)  # Add the current user to the list

    # Convert lists to numpy arrays
    data = np.array(data)
    label = np.array(label)
    users = np.array(users)  # Convert the list to a numpy array

    return data, label, users  # Return the users together with the data and labels



##Model

In [ ]:
# model define

In [ ]:
sequence_length = 100

from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

for train_indices, test_indices in k_fold.split(dataset_df.index.unique()):
  train_users, valid_users = dataset_df.index[train_indices], dataset_df.index[test_indices]
  train_x = dataset_df[dataset_df.index.isin(train_users)]
  valid_x = dataset_df[dataset_df.index.isin(valid_users)]
  model_per_question = []
  VALID_USER_LIST = valid_x.index.unique()

  prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)


  # Iterate through questions 1 to 18 to train models for each question, evaluate
  # the trained model and store the predicted values.
  for q_no in range(1,19):
      # Select level group for the question based on the q_no.
      if q_no<=4: grp = '0-4'
      elif q_no<=12: grp = '5-12'
      elif q_no<=18: grp = '13-22'
      print("### q_no", q_no, "grp", grp)

      # Filter the rows in the datasets based on the selected level group.
      train_df = train_x.loc[train_x.level_group == grp]
      train_users = train_df.index.values
      valid_df = valid_x.loc[valid_x.level_group == grp]
      valid_users = valid_df.index.values

      print(len(valid_users))

      # Select the labels for the related q_no.
      train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
      valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

      # Add the label to the filtered datasets.
      train_df.loc[:, "correct"] = train_labels["correct"]
      valid_df.loc[:, "correct"] = valid_labels["correct"]



      # Prepare data for LSTM
      train_data, train_labels, train_users = prepare_data(train_df, sequence_length)
      valid_data, valid_labels, valid_users = prepare_data(valid_df, sequence_length)





      #print(len(valid_data))
      #print(len(valid_labels))

      # Define the model
      model = Sequential()
      model.add(LSTM(64, input_shape=(sequence_length, train_data.shape[-1]), return_sequences=True))
      model.add(LSTM(32, return_sequences=False))
      model.add(Dense(1, activation='sigmoid'))

      # Compile the model
      lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.001,
      decay_steps=100,
      decay_rate=0.9
      )
      optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

      model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

      # Train the model
      if q_no in [1,5,10,11,13,15]:
        model.fit(train_data, train_labels, epochs=5, validation_data=(valid_data, valid_labels))
      else:
        model.fit(train_data, train_labels, epochs=3, validation_data=(valid_data, valid_labels))

      # Store the model
      models[f'{grp}_{q_no}'] = model

      # Evaluate the model
      # evaluation = model.evaluate(valid_data, valid_labels)
      # evaluation_dict[q_no] = evaluation[1]
      # print(f"Validation accuracy: {evaluation[1]}")

      # Make predictions
      predictions = model.predict(valid_data)
      prediction_df.loc[valid_users, q_no-1] = predictions.flatten()

      binary_predictions = [1 if p > 0.6200000000000002 else 0 for p in predictions]
      # Get the true labels
      true_labels = valid_labels[:len(valid_data)]

      # Calculate the F1 score and add it to the dictionary
      # f1 = f1_score(true_labels, binary_predictions)
      # f1_scores[q_no] = f1

  max_score = 0; best_threshold = 0

  # Create a dataframe of required size:
  # (no: of users in validation set x no: of questions) initialized to zero values
  # to store true values of the label `correct`.
  true_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)
  for i in range(18):
      # Get the true labels.
      tmp = labels.loc[labels.q == i+1].set_index('session').loc[VALID_USER_LIST]
      true_df[i] = tmp.correct.values

  # Loop through threshold values from 0.4 to 0.8 and select the threshold with
  # the highest `F1 score`.
  for threshold in np.arange(0.4,0.8,0.01):
      metric = tfa.metrics.F1Score(num_classes=2,average="macro",threshold=threshold)
      y_true = tf.one_hot(true_df.values.reshape((-1)), depth=2)
      y_pred = tf.one_hot((prediction_df.values.reshape((-1))>threshold).astype('int'), depth=2)
      metric.update_state(y_true, y_pred)
      f1_score = metric.result().numpy()
      if f1_score > max_score:
          max_score = f1_score
          best_threshold = threshold


  print("Best threshold ", best_threshold, "\tF1 score ", max_score)



### q_no 1 grp 0-4
3856


<ipython-input-45-1ee836004601>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, "correct"] = train_labels["correct"]
<ipython-input-45-1ee836004601>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.loc[:, "correct"] = valid_labels["correct"]


Epoch 1/5
244/244 [==============================] - 7s 15ms/step - loss: 0.6019 - accuracy: 0.7149 - val_loss: 0.5921 - val_accuracy: 0.7210
Epoch 2/5
244/244 [==============================] - 3s 11ms/step - loss: 0.5977 - accuracy: 0.7162 - val_loss: 0.5921 - val_accuracy: 0.7210
Epoch 3/5
244/244 [==============================] - 3s 12ms/step - loss: 0.5974 - accuracy: 0.7162 - val_loss: 0.5921 - val_accuracy: 0.7210
Epoch 4/5
244/244 [==============================] - 3s 11ms/step - loss: 0.5970 - accuracy: 0.7162 - val_loss: 0.5923 - val_accuracy: 0.7210
Epoch 5/5
121/121 [==============================] - 1s 4ms/step
### q_no 2 grp 0-4
3856


<ipython-input-45-1ee836004601>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, "correct"] = train_labels["correct"]
<ipython-input-45-1ee836004601>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.loc[:, "correct"] = valid_labels["correct"]


Epoch 1/3
244/244 [==============================] - 7s 15ms/step - loss: 0.1257 - accuracy: 0.9765 - val_loss: 0.0956 - val_accuracy: 0.9811
Epoch 2/3
244/244 [==============================] - 3s 11ms/step - loss: 0.0970 - accuracy: 0.9805 - val_loss: 0.0946 - val_accuracy: 0.9811
Epoch 3/3
121/121 [==============================] - 1s 5ms/step
### q_no 3 grp 0-4
3856


...

Epoch 1/3
244/244 [==============================] - 7s 17ms/step - loss: 0.5907 - accuracy: 0.7268 - val_loss: 0.5824 - val_accuracy: 0.7310
Epoch 2/3
244/244 [==============================] - 3s 11ms/step - loss: 0.5866 - accuracy: 0.7278 - val_loss: 0.5827 - val_accuracy: 0.7310
Epoch 3/3
121/121 [==============================] - 1s 4ms/step
### q_no 17 grp 13-22
3849


<ipython-input-45-1ee836004601>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, "correct"] = train_labels["correct"]
<ipython-input-45-1ee836004601>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.loc[:, "correct"] = valid_labels["correct"]


Epoch 1/3
244/244 [==============================] - 10s 16ms/step - loss: 0.6335 - accuracy: 0.6759 - val_loss: 0.6256 - val_accuracy: 0.6819
Epoch 2/3
244/244 [==============================] - 3s 12ms/step - loss: 0.6306 - accuracy: 0.6771 - val_loss: 0.6268 - val_accuracy: 0.6819
Epoch 3/3
121/121 [==============================] - 1s 4ms/step
### q_no 18 grp 13-22
3849


<ipython-input-45-1ee836004601>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, "correct"] = train_labels["correct"]
<ipython-input-45-1ee836004601>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df.loc[:, "correct"] = valid_labels["correct"]


Epoch 1/3
244/244 [==============================] - 7s 17ms/step - loss: 0.2211 - accuracy: 0.9459 - val_loss: 0.2047 - val_accuracy: 0.9480
Epoch 2/3
244/244 [==============================] - 3s 11ms/step - loss: 0.2001 - accuracy: 0.9498 - val_loss: 0.2063 - val_accuracy: 0.9480
Epoch 3/3
121/121 [==============================] - 1s 4ms/step
Best threshold  0.6000000000000002 	F1 score  0.64752996


In [ ]:
# Ensemble model result
LSTM_file_path ='/content/drive/MyDrive/9417project/predictions/LSTM_pred.npy'
SVM_file_path ='/content/drive/MyDrive/9417project/predictions/SVM_pred.npy'
np.save(LSTM_file_path,prediction_df)
LSTM_pred = np.load(LSTM_file_path,allow_pickle=True)
